# 🗺️ Procedural Fantasy WorldGen — v7 (Modular YAML)

This version loads **all `.yaml` files** in a folder, deep-merges them (later files override earlier ones),
and then runs the same world generators.

Place your world fragments under a directory (default: `/mnt/data/world_mod`).


In [10]:
pip install pyyaml  # Uncomment if needed

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import re, random, math, json, os, glob
from copy import deepcopy
from datetime import datetime
from typing import Any, Dict, List, Optional
from functools import reduce
import yaml

# Constants for magic strings
PROCGEN_TABLES = 'procgen.tables'
PROCGEN_GENERATORS = 'procgen.generators'
PROCGEN_WEIGHTS = 'procgen.weights'
PROCGEN_DICE = 'procgen.dice'

# Compiled regex patterns (module-level for performance)
DICE_PAT = re.compile(r"(?:(\d+)[dD](\d+))|([()+\-*/])|(\d+)")
CURLY_RE = re.compile(r"\{\{\s*([a-zA-Z0-9_.-]+)(\|[a-zA-Z]+)?\s*\}\}")

# Global state (will be replaced by context object)
CURRENT_ROOT = None
CURRENT_RNG = None
_TABLE_CACHE = {}

def make_rng(seed: Optional[int]) -> random.Random:
    """Create a random number generator with optional seed."""
    rng = random.Random()
    rng.seed(seed if seed is not None else None)
    return rng

def deep_merge(a: dict, b: dict) -> dict:
    """Deep merge two dictionaries. Optimized to avoid double deep-copying."""
    out = a.copy()  # Shallow copy first
    for k, v in b.items():
        if k in out and isinstance(out[k], dict) and isinstance(v, dict):
            out[k] = deep_merge(out[k], v)
        else:
            out[k] = deepcopy(v)  # Only deep copy leaf values
    return out

def load_yaml_file(path: str) -> dict:
    """Load a single YAML file."""
    with open(path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f) or {}

def load_world_folder(folder: str) -> dict:
    """Load and merge all YAML files in a folder. Optimized with reduce."""
    files = sorted(glob.glob(os.path.join(folder, '*.yaml')))
    if not files:
        return {}
    fragments = [load_yaml_file(p) for p in files]
    return reduce(deep_merge, fragments, {})

def _roll_single(rng: random.Random, ndice: int, sides: int) -> int:
    """Roll ndice with given number of sides."""
    return sum(rng.randint(1, sides) for _ in range(ndice))

def roll(expr: str, rng: Optional[random.Random] = None) -> int:
    """Parse and evaluate a dice expression like '2d6+3'."""
    if not expr:
        raise ValueError("Empty dice expression")
    if rng is None:
        rng = CURRENT_RNG
    tokens = []
    for m in DICE_PAT.finditer(str(expr)):
        if m.group(1) and m.group(2):
            nd, sd = int(m.group(1)), int(m.group(2))
            tokens.append(str(_roll_single(rng, nd, sd)))
        elif m.group(3):
            tokens.append(m.group(3))
        elif m.group(4):
            tokens.append(m.group(4))
    safe = ''.join(tokens)
    try:
        return int(eval(safe, {"__builtins__": {}}, {}))
    except Exception as e:
        raise ValueError(f"Bad dice expr '{expr}' -> '{safe}': {e}")

def weighted_choice(weights: Dict[str, int], rng: random.Random) -> str:
    """Choose a random key from weights dict. Optimized with random.choices."""
    if not weights:
        raise ValueError("Empty weights dictionary")
    items = list(weights.keys())
    wts = [max(int(w), 0) for w in weights.values()]
    total = sum(wts)
    if total <= 0:
        raise ValueError("Weights sum to zero")
    return rng.choices(items, weights=wts, k=1)[0]

def deep_get(d: Dict[str, Any], path: str) -> Any:
    """Navigate nested dict using dot-separated path."""
    cur = d
    for part in path.split('.'):
        if part == '':
            continue
        if isinstance(cur, dict) and part in cur:
            cur = cur[part]
        else:
            raise KeyError(f"Path '{path}' not found (stuck at '{part}')")
    return cur

def interpolate(s: str, ctx: Dict[str, Any]) -> str:
    """Replace {{key}} and {{key|filter}} patterns with context values."""
    def repl(m):
        key = m.group(1)
        filt = (m.group(2) or '').lstrip('|')
        val = ctx
        for p in key.split('.'):
            if isinstance(val, dict) and p in val:
                val = val[p]
            else:
                return m.group(0)
        sval = str(val)
        if   filt == 'title': sval = sval.title()
        elif filt == 'upper': sval = sval.upper()
        elif filt == 'lower': sval = sval.lower()
        return sval
    return CURLY_RE.sub(repl, s)

def case(key: Any, mapping: Dict[str, Any], default=None):
    """Simple case/switch statement helper."""
    return mapping.get(str(key), default)

def treasure_tier_by_cr(cr_val: Any) -> Any:
    """Look up treasure tier based on CR value."""
    if CURRENT_ROOT is None:
        raise RuntimeError("CURRENT_ROOT not set")
    try:
        cr = float(cr_val)
    except Exception:
        cr = 0.0
    table = deep_get(CURRENT_ROOT, f'{PROCGEN_DICE}.treasure_tier_by_cr')
    
    # Single-pass lookup with priority: exact match > range > plus
    exact_match = None
    range_match = None
    plus_match = None
    
    for k, v in table.items():
        k_str = str(k).strip()
        
        # Check for exact match
        try:
            if float(k_str) == cr:
                exact_match = v
                break
        except ValueError:
            pass
        
        # Check for range (e.g., "3-6")
        if '-' in k_str:
            try:
                a, b = k_str.split('-', 1)
                if float(a) <= cr <= float(b):
                    range_match = v
            except ValueError:
                pass
        
        # Check for plus (e.g., "15+")
        if k_str.endswith('+'):
            try:
                start = float(k_str[:-1])
                if cr >= start:
                    plus_match = v
            except ValueError:
                pass
    
    return exact_match or range_match or plus_match or next(iter(table.values()))

def lookup(path: str) -> Any:
    """Look up a value in the world root by path."""
    if CURRENT_ROOT is None:
        raise RuntimeError("CURRENT_ROOT not set")
    return deep_get(CURRENT_ROOT, path)

class SafeLocals(dict):
    """Safe locals dict for eval that includes context variables."""
    RESERVED = {"treasure_tier_by_cr","lookup","roll","case","math",
                "max","min","abs","round","int","float","len","sum"}
    
    def __init__(self, base_ctx: Dict[str, Any]):
        super().__init__()
        self._ctx = base_ctx
    
    def __contains__(self, key):
        return dict.__contains__(self, key) or (key in self._ctx and key not in self.RESERVED)
    
    def __getitem__(self, key):
        if dict.__contains__(self, key):
            return dict.__getitem__(self, key)
        if key in self._ctx and key not in self.RESERVED:
            return self._ctx[key]
        raise KeyError(key)
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default

def eval_expr(expr: str, ctx: Dict[str, Any]) -> Any:
    """Safely evaluate an expression with context."""
    safe_locals = SafeLocals(ctx)
    safe_locals.update({
        "case": case,
        "math": math,
        "treasure_tier_by_cr": treasure_tier_by_cr,
        "lookup": lookup,
        "roll": lambda s: roll(s, CURRENT_RNG),
        "max": max, "min": min, "abs": abs, "round": round,
        "int": int, "float": float, "len": len, "sum": sum,
    })
    try:
        return eval(expr, {"__builtins__": {}}, safe_locals)
    except Exception as e:
        raise ValueError(f"Bad derive/bind expr '{expr}': {e}")

def resolve_at(ref: str, root: Dict[str, Any], runtime: Dict[str, Any]) -> Any:
    """Resolve @-prefixed references to world data."""
    if not ref.startswith('@'):
        return ref
    path = ref[1:]
    if path.startswith('weights.'):
        path = f'{PROCGEN_WEIGHTS}.{path[8:]}'
    elif path.startswith('dice.'):
        path = f'{PROCGEN_DICE}.{path[5:]}'
    
    # Conditional interpolation - only if needed
    if '{{' in path:
        path = interpolate(path, runtime)
    
    if '[' in path and path.endswith(']'):
        head, key = path.split('[', 1)
        key = key[:-1]
        base = deep_get(root, head)
        return base[key]
    return deep_get(root, path)

def build_table_cache(root: Dict[str, Any]) -> None:
    """Build lookup cache for procgen tables."""
    global _TABLE_CACHE
    try:
        tables = deep_get(root, PROCGEN_TABLES)
        _TABLE_CACHE = {t['id']: t for t in tables if 'id' in t}
    except KeyError:
        _TABLE_CACHE = {}

def get_proc_table(root: Dict[str, Any], table_id: str) -> Dict[str, Any]:
    """Get a procgen table by ID. Uses cache for performance."""
    if not _TABLE_CACHE:
        build_table_cache(root)
    if table_id not in _TABLE_CACHE:
        raise KeyError(f"Table '{table_id}' not found in {PROCGEN_TABLES}")
    return _TABLE_CACHE[table_id]

def roll_table(table: Dict[str, Any], rng: random.Random) -> Any:
    """Roll on a weighted table and return a result."""
    entries = table.get('entries', [])
    if not entries:
        return None
    total = sum(int(e.get('weight', 1)) for e in entries)
    r = rng.uniform(0, total)
    acc = 0
    for e in entries:
        acc += int(e.get('weight', 1))
        if r <= acc:
            return e.get('result', e)
    return entries[-1].get('result', entries[-1])

def run_generator(root: Dict[str, Any], gen_name: str, rng: random.Random, 
                  base_ctx: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """Run a named generator and return the resulting context."""
    gens = deep_get(root, PROCGEN_GENERATORS)
    if gen_name not in gens:
        raise KeyError(f"Generator '{gen_name}' not found in {PROCGEN_GENERATORS}")
    steps = gens[gen_name].get('steps', [])
    ctx = {} if base_ctx is None else deepcopy(base_ctx)
    ctx['now'] = datetime.utcnow().isoformat()

    for step in steps:
        if 'choose' in step:
            spec = step['choose']
            for k, v in spec.items():
                if isinstance(v, str) and v.startswith('@'):
                    resolved = resolve_at(v, root, ctx)
                    if isinstance(resolved, dict):
                        ctx[k] = weighted_choice(resolved, rng)
                    else:
                        ctx[k] = resolved
                else:
                    ctx[k] = v

        elif 'roll' in step:
            spec = step['roll']
            for k, v in spec.items():
                if isinstance(v, str) and v.startswith('@'):
                    resolved = resolve_at(v, root, ctx)
                    expr = resolved
                    ctx[k] = roll(str(expr), rng)
                else:
                    ctx[k] = roll(str(v), rng)

        elif 'pick_n' in step:
            spec = step['pick_n']
            src = spec.get('from', [])
            n_expr = spec.get('n', 1)
            n = roll(str(n_expr), rng) if isinstance(n_expr, str) else int(n_expr)
            pool = list(src)
            rng.shuffle(pool)
            ctx['problems'] = pool[:n]

        elif 'derive' in step:
            spec = step['derive']
            for k, expr in spec.items():
                ctx[k] = eval_expr(str(expr), ctx)

        elif 'table' in step:
            table_id = step['table']
            t = get_proc_table(root, table_id)
            res = roll_table(t, rng)
            ctx[table_id] = res
            alias = table_id.split('_')[0]
            if alias and alias not in ctx:
                ctx[alias] = res

        elif 'bind' in step:
            spec = step['bind']
            for k, expr in spec.items():
                ctx[k] = eval_expr(str(expr), ctx) if isinstance(expr, str) else expr

        elif 'ensure' in step:
            pass

        elif 'output_template' in step:
            tmpl = step['output_template']
            ctx['output'] = interpolate(tmpl, ctx)

    return ctx

def list_generators(root: Dict[str, Any]) -> List[str]:
    """List all available generator names."""
    gens = deep_get(root, PROCGEN_GENERATORS)
    return sorted(gens.keys())

In [12]:
from pprint import pprint

WORLD_DIR = '/mnt/data/world_mod'  # change if needed
root = load_world_folder(WORLD_DIR)

seed = (root.get('meta', {}) or {}).get('seed')
rng = make_rng(seed)
CURRENT_ROOT = root
CURRENT_RNG = rng

print('Loaded folder:', WORLD_DIR)
print('Merged keys:', list(root.keys()))
print('Seed:', seed)
print('Generators:', ', '.join(list_generators(root)))

Loaded folder: /mnt/data/world_mod
Merged keys: []
Seed: None


KeyError: "Path 'procgen.generators' not found (stuck at 'procgen')"

In [0]:
# Run sample generators and save outputs
samples_to_run = ['settlement', 'dungeon', 'quest']
outputs = {}
for gname in samples_to_run:
    try:
        outputs[gname] = run_generator(root, gname, rng)
    except KeyError as e:
        outputs[gname] = {"error": str(e)}

from pprint import pprint
pprint(outputs)

for gname, data in outputs.items():
    out = data.get('output') if isinstance(data, dict) else None
    if out:
        print(f"\n--- {gname.upper()} TEMPLATE OUTPUT ---\n{out}")

save_path = '/mnt/data/worldgen_outputs.json'
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(outputs, f, ensure_ascii=False, indent=2)
print('\nSaved JSON:', save_path)

quest_out = outputs.get('quest', {}).get('output') if isinstance(outputs.get('quest'), dict) else None
if quest_out:
    qpath = '/mnt/data/worldgen_quest.txt'
    with open(qpath, 'w', encoding='utf-8') as f:
        f.write(quest_out)
    print('Saved quest text:', qpath)